# Run fitsverify

In [1]:
import os
import sys
import re
import subprocess as sp
from configparser import ConfigParser
from random import choice
specprod = 'everest'
specprod_path = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)

## Create input file

In [2]:
fits_files = os.path.join(os.environ['CSCRATCH'], f'{specprod}_fits.txt')
if not os.path.exists(fits_files):
    # os.chdir(specprod_path)
    with open(fits_files, 'w') as out:
        command = ['find', 'calibnight', 'exposures', 'healpix', 'preproc', 'tiles', 'zcatalog', '-type', 'f', '-name', '*.fits']
        proc = sp.Popen(command, stdout=out, stderr=sp.DEVNULL, cwd=specprod_path)
        status = proc.wait()

## List of Regular Expressions

In [3]:
parser = ConfigParser()
parser.read_string("""
[top]
exposures = exposures-everest\.fits

[calibnight]
fiberflatnight = calibnight/[0-9]{8}/fiberflatnight-[brz][0-9]-[0-9]{8}\.fits
psfnight = calibnight/[0-9]{8}/psfnight-[brz][0-9]-[0-9]{8}\.fits

[exposures]
cframe = exposures/[0-9]{8}/[0-9]{8}/cframe-[brz][0-9]-[0-9]{8}\.fits
exposure-qa = exposures/[0-9]{8}/[0-9]{8}/exposure-qa-[0-9]{8}\.fits
fiberflat = exposures/[0-9]{8}/[0-9]{8}/fiberflat-[brz][0-9]-[0-9]{8}\.fits
fit-psf = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}\.fits
fit-psf-before-blacklisted = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-blacklisted-[brz][0-9]-[0-9]{8}\.fits
fit-psf-before-blacklisted-fix = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-blacklisted-fix-[brz][0-9]-[0-9]{8}\.fits
fit-psf-fixed-blacklisted = exposures/[0-9]{8}/[0-9]{8}/fit-psf-fixed-blacklisted-[brz][0-9]-[0-9]{8}\.fits
fluxcalib = exposures/[0-9]{8}/[0-9]{8}/fluxcalib-[brz][0-9]-[0-9]{8}\.fits
frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}\.fits
psf = exposures/[0-9]{8}/[0-9]{8}/psf-[brz][0-9]-[0-9]{8}\.fits
sframe = exposures/[0-9]{8}/[0-9]{8}/sframe-[brz][0-9]-[0-9]{8}\.fits
shifted-input-psf = exposures/[0-9]{8}/[0-9]{8}/shifted-input-psf-[brz][0-9]-[0-9]{8}\.fits
sky = exposures/[0-9]{8}/[0-9]{8}/sky-[brz][0-9]-[0-9]{8}\.fits
stdstars = exposures/[0-9]{8}/[0-9]{8}/stdstars-[0-9]-[0-9]{8}\.fits

[healpix]
coadd = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/coadd-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
qso_mgii = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_mgii-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
qso_qn = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_qn-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
redrock = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/redrock-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
spectra = healpix/(sv1|sv2|sv3|main)/(backup|bright|dark|other)/[0-9]+/[0-9]+/spectra-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-[0-9]+\.fits
tilepix = healpix/tilepix\.fits

[preproc]
fibermap = preproc/[0-9]{8}/[0-9]{8}/fibermap-[0-9]{8}\.fits
preproc = preproc/[0-9]{8}/[0-9]{8}/preproc-[brz][0-9]-[0-9]{8}\.fits

[tiles]
coadd = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/coadd-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
qso_mgii = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_mgii-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
qso_qn = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_qn-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
redrock = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/redrock-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
spectra = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/spectra-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits
tile-qa = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.fits

[tiles:depth]
coadd = tiles/[14]x_depth/[0-9]+/[0-9]/coadd-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
qso_mgii = tiles/[14]x_depth/[0-9]+/[0-9]/qso_mgii-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
qso_qn = tiles/[14]x_depth/[0-9]+/[0-9]/qso_qn-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
redrock = tiles/[14]x_depth/[0-9]+/[0-9]/redrock-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits
spectra = tiles/[14]x_depth/[0-9]+/[0-9]/spectra-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits

[zcatalog]
zpix = zcatalog/zpix-(sv1|sv2|sv3|main)-(backup|bright|dark|other)\.fits
ztile = zcatalog/ztile-(sv1|sv2|sv3|main)-(backup|bright|dark|other)-(cumulative|pernight)\.fits

[exclude]
calibnight = calibnight/[0-9]{8}/tmp/*
exposures = exposures/[0-9]{8}/old/*
preproc = preproc/[0-9]{8}/old/*
"""
    )


## Precompile Regular Expressions

In [5]:
r = dict()
for s in parser.sections():
    r[s] = dict()
    for key, value in parser.items(s):
        r[s][key] = re.compile(value)

## Scan the list of files

In [6]:
with open(fits_files) as e:
    data = e.readlines()
data.append(f'exposures-{specprod}.fits\n')
scanable = dict()
for file in data:
    ff = file.strip()
    f = ff.split('/')
    if len(f) == 1:
        section = 'top'
    elif f[1] == '1x_depth' or f[1] == '4x_depth':
        section = 'tiles:depth'
    else:
        section = f[0]
    if section not in scanable:
        scanable[section] = dict()
    excluded = False
    for key in r['exclude']:
        m = r['exclude'][key].match(ff)
        if m is not None:
            excluded = True
    if excluded:
        continue
    matched = False
    for key in r[section]:
        m = r[section][key].match(ff)
        if m is not None:
            matched = True
            if key in scanable[section]:
                scanable[section][key].append(ff)
            else:
                scanable[section][key] = [ff]
    if matched:
        continue
    print("ERROR: Could not match {0}!".format(ff))

## From the list of all file types, pick one at random

In [7]:
scan = dict()
for section in scanable:
    scan[section] = dict()
    for ftype in scanable[section]:
        scan[section][ftype] = choice(scanable[section][ftype])
scan

{'calibnight': {'fiberflatnight': 'calibnight/20210317/fiberflatnight-r1-20210317.fits',
  'psfnight': 'calibnight/20210521/psfnight-r2-20210521.fits'},
 'exposures': {'frame': 'exposures/20210610/00093280/frame-r2-00093280.fits',
  'psf': 'exposures/20210108/00071485/psf-b8-00071485.fits',
  'fiberflat': 'exposures/20201218/00068558/fiberflat-b2-00068558.fits',
  'sky': 'exposures/20210318/00080968/sky-z8-00080968.fits',
  'sframe': 'exposures/20210223/00077765/sframe-b6-00077765.fits',
  'cframe': 'exposures/20210102/00070567/cframe-r8-00070567.fits',
  'fluxcalib': 'exposures/20210205/00074787/fluxcalib-b2-00074787.fits',
  'stdstars': 'exposures/20210506/00087489/stdstars-2-00087489.fits',
  'shifted-input-psf': 'exposures/20210411/00084304/shifted-input-psf-b3-00084304.fits',
  'fit-psf': 'exposures/20210223/00077648/fit-psf-z7-00077648.fits',
  'fit-psf-before-blacklisted-fix': 'exposures/20210616/00094850/fit-psf-before-blacklisted-fix-b3-00094850.fits',
  'fit-psf-fixed-blackli

## Run fitsverify on the chosen files

In [17]:
for section in scan:
    for key in scan[section]:
        command = ['fitsverify', '-l', scan[section][key]]
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
        out, err = proc.communicate()
        print(section, key, out.decode('ascii').split('\n')[-2])
        if err:
            print(err.decode('ascii'))

calibnight fiberflatnight **** Verification found 3 warning(s) and 0 error(s). ****
calibnight psfnight **** Verification found 0 warning(s) and 0 error(s). ****
exposures frame **** Verification found 12 warning(s) and 5 error(s). ****
*** Error:   Keyword #233, SIMPLE is not allowed in extensions.
*** Error:   Keyword #610, EXTEND is not allowed in extensions.
*** Error:   Keyword #235, BSCALE is not allowed in the Bin/ASCII table.
*** Error:   Keyword #234, BZERO is not allowed in the Bin/ASCII table.
*** Error:   Keyword #233, SIMPLE is duplicated or out of order.

exposures psf **** Verification found 0 warning(s) and 0 error(s). ****
exposures fiberflat **** Verification found 6 warning(s) and 4 error(s). ****
*** Error:   Keyword #349, SIMPLE is not allowed in extensions.
*** Error:   Keyword #351, BSCALE is not allowed in the Bin/ASCII table.
*** Error:   Keyword #350, BZERO is not allowed in the Bin/ASCII table.
*** Error:   Keyword #349, SIMPLE is duplicated or out of order.
